In [10]:
from typing import List, Dict
from csv import DictReader

def open_data(filename: str) -> List[Dict[str, str]]:
    """
    Opens CSV file, returns entries
    """

    with open(filename, 'r') as csv_file:
        reader = DictReader(csv_file)
        rows = [r for r in reader]
    csv_file.close()
    return rows

def prune_data(rows: List[Dict[str, str]]) -> List[Dict[str, float]]:
    """
    Removes data samples with blank values in input/target data
    """

    new_rows = []
    keys = list(rows[0].keys())
    for row in rows:
        new_row = {}
        _valid = True
        for k in keys:
            try:
                new_row[k] = float(row[k])
            except ValueError:
                _valid = False
                break
        if _valid:
            new_rows.append(new_row)
    return new_rows

data_fb = open_data('_data/fluidized_bed.csv')
data_fb = prune_data(data_fb)
data_tr = open_data('_data/tubular_reactor.csv')
data_tr = prune_data(data_tr)
data_br = open_data('_data/batch_reactor.csv')
data_br = prune_data(data_br)

print(len(data_fb), len(data_tr), len(data_br))

7 10 27


In [11]:
from structs import ReactorDataset

def create_dataset(data: List[Dict[str, float]], target_headers: List[str]) -> 'structs.ReactorDataset':

    input_headers = list(data[0].keys())
    for head in target_headers:
        input_headers.remove(head)

    X = []
    y = []

targets = ['water', 'Oil', 'Char', 'Gas']